## Tugas

1. Menggunakan data (WBC) dan alogirtma yang sama (DT) pada jobsheet js11_02, silahkan implemnetasikan ulang teknik *exhausive search* dengan menggunakan ***random search***. Scikit-learn sudah menyediakan fungsi ini.

2. Menggunakan data yang sama, lakukan tunning dengan menggunakan algoritma SVM. Hyperparamter yang akan ditunning adalah,
    - kernel (linier, polinomial, rbf)
    - degree (akan berdampak jika polinomial)
    - koefisien C (regularization)

### Persiapan Data

In [4]:
import numpy as np
import pandas as pd

# Load data
df = pd.read_csv('data/wbc.csv')

# slice kolom terakhir --> Unnamed column
df = df.iloc[:,:-1]
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### Seleksi fitur

In [5]:
X = df.iloc[:, 2:]
y = df['diagnosis']

X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### Split data training dan testing

In [6]:
# Implementasi repeated k-fold cross validation data training, validasi, dan testing dengan stratified
from sklearn.model_selection import train_test_split, RepeatedKFold

# Split train test dengan stratified
df_train, df_test = train_test_split(df, test_size=0.8, random_state=0, stratify=df['diagnosis'])


# inisiasi obyek RepeatedKFold
rkf = RepeatedKFold(n_splits=4, n_repeats=3) # n_repeats digunakan untuk mendefinisikan berapa kali ulangan
print(f'Jumlah fold: {rkf.get_n_splits()}')
print(f'Obyek SKFold: {rkf}')

# Lakukan splitting dengan RKFold
# parameter kedua dari split adalah labelnya
rkf_split = rkf.split(df_train, df_train['diagnosis'])
print(f'Jumlah data df: {df_train.shape[0]}')

# cek index data tiap fold
for train_index, test_index in rkf_split:
    print(f'Index train: {train_index} | Index test: {test_index}')

Jumlah fold: 12
Obyek SKFold: RepeatedKFold(n_repeats=3, n_splits=4, random_state=None)
Jumlah data df: 113
Index train: [  0   1   2   3   5   8   9  10  11  15  16  18  19  20  21  22  23  24
  25  27  28  29  31  32  33  34  35  36  37  38  40  41  43  44  45  46
  47  48  50  51  53  54  56  58  59  60  61  63  64  65  66  68  69  72
  73  74  75  76  79  80  81  82  83  85  86  88  89  90  92  94  95  97
  98 100 103 104 105 106 107 108 109 110 111 112] | Index test: [  4   6   7  12  13  14  17  26  30  39  42  49  52  55  57  62  67  70
  71  77  78  84  87  91  93  96  99 101 102]
Index train: [  1   2   4   5   6   7   8   9  11  12  13  14  15  16  17  19  21  22
  23  25  26  27  28  30  31  33  35  36  37  39  40  42  43  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  64  66  67
  68  69  70  71  72  74  77  78  80  81  82  83  84  87  88  89  91  93
  96  97  99 100 101 102 103 104 105 106 107 108 112] | Index test: [  0   3  10  18  20  24  29  3

## Implementasi Random Search

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

# inisiasi model
dt = DecisionTreeClassifier()

# Definisikan hyperparameter yang akan digunakan
# sklearn menerima dalam bentuk dictionary
# nama hyperparamater HARUS SESUAI dengan dokumentasi sklearn
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': list(range(5,11))
}

# inisiasi random search berdasarkan nilai repeated k-fold dan hyperparameter
random = RandomizedSearchCV(dt, param_distributions=params, n_jobs=-1, cv=rkf)

# Fit / latih berdasarkan grid
# %timeit merupakan magic command didalam ipython notebook
# yang dapat kita gunakan untuk menghitung waktu komputasi
# cara ini cukup efektif untuk melakukan evaluasi suatu algoritma atau prosedur
%timeit random.fit(X, y)

# Evaluasi dengan score
score = random.score(X,y)

print(f'Hasil evaluasi: {score}')
print(f'Konfigurasi hyperparameter: {random.best_params_}')

1.61 s ± 199 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Hasil evaluasi: 1.0
Konfigurasi hyperparameter: {'max_depth': 8, 'criterion': 'log_loss'}


## Tuning menggunakan algoritma SVM

In [8]:
# import library
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

svc = SVC()

# Definisikan hyperparameter yang akan digunakan
# sklearn menerima dalam bentuk dictionary
# nama hyperparamater HARUS SESUAI dengan dokumentasi sklearn
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [1, 5, 10, 50, 100],
    'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

# inisiasi random search berdasarkan nilai repeated k-fold dan hyperparameter
random_search = RandomizedSearchCV(svc, param_distributions=param_grid, n_jobs=-1, cv=rkf)

# Fit / latih berdasarkan grid
# %timeit merupakan magic command didalam ipython notebook
# yang dapat kita gunakan untuk menghitung waktu komputasi
# cara ini cukup efektif untuk melakukan evaluasi suatu algoritma atau prosedur
%timeit random_search.fit(X, y)

# Evaluasi dengan score
score2 = random_search.score(X,y)

print(f'Hasil evaluasi: {score2}')
print(f'Konfigurasi hyperparameter: {random_search.best_params_}')

The slowest run took 8.73 times longer than the fastest. This could mean that an intermediate result is being cached.
2min 36s ± 1min 21s per loop (mean ± std. dev. of 7 runs, 1 loop each)
Hasil evaluasi: 0.9666080843585237
Konfigurasi hyperparameter: {'kernel': 'linear', 'degree': 3, 'C': 1}
